In [1]:
import numpy as np 
import pandas as pd

In [2]:
def impute_from_column(df, impute_to, impute_from):
    """
    test:
    df[(df['c_ssrs_6'].isna()) & (~df['c_ssrs_last_visit_6'].isna())][['c_ssrs_6', 'c_ssrs_last_visit_6']]
    df = impute_from_column(df, impute_to = 'c_ssrs_6', impute_from = 'c_ssrs_last_visit_6')
    df[(df['c_ssrs_6'].isna()) & (~df['c_ssrs_last_visit_6'].isna())][['c_ssrs_6', 'c_ssrs_last_visit_6']]
    
    """
    df[impute_to] = np.where(df[impute_to].isnull(), df[impute_from], df[impute_to])
    
    return df

df = pd.read_csv(r'../preprocessed_data/merged_2021_and_2022.csv', na_values = ' ')

C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,44,50,73,84,309,437,442,443,450,457,486,502,540,549,568,728,733,745,747,752,763,801,810,829,882,883,884,885,886,887,888,889,890,891,892,951,989,994,1000,1015,1045,1065,1066,1070,1071,1072,1073,1075,1076,1077,1080,1081,1098,1103,1119,1121,1172,1174,1193,1195,1196,1197,1203,1204,1205,1206,1358,1360,1361,1362,1368,1369,1370,1371,1486,1492,1493,1496,1497,1498,1499,1508,1516,1532,1535,1546) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
cols = list(df.columns)
id_column = 'id'
cols.remove('redcap_event_name')
cols.remove('age_child')
limit = cols.index('age_child_pre')
duplicate_cols = list(cols[limit + 1:])
unique_cols = list(cols[:limit + 1])
all_columns = unique_cols + duplicate_cols

In [4]:
df_intake = df[df.redcap_event_name == 'intake_arm_1'][all_columns]
df_pre = df[df.redcap_event_name == 'pre_treatment_arm_1'][all_columns]#[duplicate_cols + [id_column]]

In [5]:
merged = pd.merge(df_intake, df_pre, on=[id_column], how='outer', suffixes=('', '_pre_treatment_arm_1'))

In [6]:
for column_name in [i for i in all_columns if i != 'id']: #duplicate_cols:
    merged = impute_from_column(merged, impute_to = column_name, impute_from = f'{column_name}_pre_treatment_arm_1')

In [7]:
pre_treatment_cols = [i for i in merged.columns if i.endswith('_pre_treatment_arm_1')]
merged = merged.drop(pre_treatment_cols, axis=1)

In [8]:
merged.to_csv("intake_data.csv", index=False)

In [9]:
merged.shape

(419, 1545)

In [10]:
a = (merged.groupby('id').age_child_pre.count() == 0)

In [11]:
a[a].sum()

16

In [12]:
a[a].to_csv("missing_age_part_2.csv")